# Machine Learning Project: Key Detection
Authors: Zach Hayes, Matt Gaetano, Max Ivry

Course: CS3540


In [ ]:
import pandas as pd
import numpy as np


### Data Cleaning and Feature Engineering Pipeline

In [ ]:
def json_to_df():
    """
    Convert the json data file to a dataframe.
    """

In [ ]:
def clean_df(df):
    """
    Remove irrelavent features from dataframe.
    Remove instances from dataframe below key
    certainty threshold.
    """

In [ ]:
def features_from_audio(df):
    """
    'Join' the existing dataframe with the audio files via
    mapping to spectrograph features.
    """

In [3]:
def df_pipeline():
    """
    From the original audio file and json pair dataset, 
    create the dataset from which to work 
    """
    df = json_to_df()
    clean_df(df)
    features_from_audio(df)
    return df

df = df_pipeline()

### Support Vector Machine Approach